> A project to analyze hacker news stories using nbdbt

## Analysis

In [1]:
%reload_ext nbdbt.dbt_cellmagic

In [2]:
%dbtconfig -p ../hn_whos_hiring

In [3]:
%%dbt -a raw_sources -n notebooks/analysis.ipynb analyses/raw_hn_source.sql
select *
from {{ source('public_datasets', 'full_stories') }}
order by timestamp desc
